In [5]:
from models.model_handler import *
from data_processing.data_pipeline import *
import torch
import torch.nn as nn
import torchvision.models as models

In [2]:
class CNNModel(nn.Module):
    """A convolutional neural network model based on ResNet for spectrogram processing.

    Attributes:
        input_folder (str): Path to the input folder containing spectrograms.
        output_folder (str): Path to the folder where model outputs will be saved.
        resnet (torchvision.models.resnet.ResNet): The ResNet backbone used for feature extraction.
    """

    def __init__(self, dropout: float = 0.0):
        """Initializes the CNNModel.
        """
        super(CNNModel, self).__init__()
        self.resnet = models.resnet18(weights='IMAGENET1K_V1')
        num_features = self.resnet.fc.in_features  # Get input size of original FC layer

        # Freeze all the pre-trained layers
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # Remove the last FC layer and replace it with a binary classifier
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=dropout),  # Apply dropout before the final layer
            nn.Linear(num_features, 1)  # Binary classification output
        )

        # Initialize weights and biases for the new FC layer
        self.resnet.fc[1].weight.data.normal_(mean=0.0, std=0.01)
        self.resnet.fc[1].bias.data.zero_()

    def forward(self, spectrogram: torch.Tensor) -> torch.Tensor:
        """Defines the forward pass for the model.

        Args:
            spectrogram (torch.Tensor): Input tensor representing the spectrogram.

        Returns:
            torch.Tensor: The model's output after processing the spectrogram.
        """
        return self.resnet(spectrogram)


In [3]:
DROPOUT = 0

model = models.resnet18(weights='IMAGENET1K_V1')

print(model.fc.in_features)

model.fc = nn.Sequential(
    nn.Dropout(DROPOUT),
    nn.Linear(512, 1)
)



512


In [8]:
from pathlib import Path

original_audio = Path("/home/yoshikuyuu/Projects/RespiraCheck/ml/data/cough_data/original_data")
processed_audio = Path("/home/yoshikuyuu/Projects/RespiraCheck/ml/data/cough_data/processed_audio")
spectrograms = Path("/home/yoshikuyuu/Projects/RespiraCheck/ml/data/cough_data/spectrograms")

audioproccessor = AudioProcessor(input_folder=original_audio,
                                 output_folder=processed_audio)
spectroproccessor = SpectrogramProcessor(audio_folder=processed_audio,
                                         output_folder=spectrograms)

datapipeline = DataPipeline(test_size=0.15, 
                                val_size=0.15, 
                                audio_processor=audioproccessor, 
                                image_processor=spectroproccessor)

cnn_model = CNNModel()
loss_function = nn.BCEWithLogitsLoss()

# optimizer = torch.optim.SGD(params=cnn_model.parameters(), lr=0.01, momentum=0.9) ###SDG
optimizer = torch.optim.Adam(params=cnn_model.parameters(), lr=0.01) ### ADAM

model_handler = ModelHandler(model=cnn_model, model_path="ml/models", optimizer=optimizer, loss_function=loss_function)

# if not os.path.exists("data/cough_data/spectrograms"):
#     datapipeline.process_all()
# train_loader, val_loader, test_loader = datapipeline.create_dataloaders(batch_size=32)

In [9]:
datapipeline.image_processor.output_folder

PosixPath('/home/yoshikuyuu/Projects/RespiraCheck/ml/data/cough_data/spectrograms')

In [ ]:
# Train the model
epochs = 1

model_handler.train(train_loader=train_loader, epochs=epochs, model_name="g1_model")

best_model = None
best_acc = 0.0

# Hyperparameters for validation
hyperparameter_options = [
    {"learning_rate": 0.01},
    {"learning_rate": 0.001},
    {"learning_rate": 0.0001}
]

for hyperparams in hyperparameter_options:
    print(f"Validating model with hyperparameters: {hyperparams}")

    cnn_model = CNNModel()
    # optimizer = torch.optim.SGD(params=cnn_model.parameters(), lr=hyperparams["learning_rate"], momentum=0.9) ###SDG
    optimizer = torch.optim.Adam(params=cnn_model.parameters(), lr=0.01) ### ADAM

    # Create new ModelHandler for each hyperparameter set
    model_handler = ModelHandler(model=cnn_model, model_path="ml/models", optimizer=optimizer, loss_function=loss_function)
    
    # Perform validation
    val_acc, val_loss = model_handler.validate(val_loader, hyperparams)

    # Save the best model based on accuracy
    if val_acc > best_acc:
        best_acc = val_acc
        best_model = model_handler

    print(f"Validation accuracy: {val_acc*100:.2f}% | Validation loss: {val_loss:.4f}")

# Final testing with the best model
if best_model:
    test_acc = best_model.evaluate(test_loader)
    print(f"Test accuracy: {test_acc*100:.2f}%. Awesome!")